In [1]:
import kaggle
import pandas as pd
import os
import lightgbm as lgbm
import optuna as opt
import seaborn as sns
import numpy as np
import gc
import shap
from scipy.stats import binom_test
from sklearn.model_selection import train_test_split

In [2]:
os.chdir("/Users/matthewlafferty/Dropbox/Kaggle/Tabular_Feb_2021/tabular-playground-series-feb-2021")

In [3]:
data = pd.read_csv('train.csv')

In [4]:
data.sample(10)

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
262658,437948,B,A,A,A,B,B,A,E,C,...,0.244489,0.689357,0.637401,0.483618,0.297313,0.283682,0.852316,0.285454,0.701334,7.515493
113751,189563,A,A,A,C,B,D,A,E,C,...,0.583045,0.930331,0.300816,0.127790,0.224117,0.672991,0.351220,0.358266,0.775926,6.509431
190510,317720,A,A,A,A,B,D,A,E,E,...,0.862581,0.729128,0.926942,0.880656,0.810298,0.849737,0.799923,0.753010,0.822090,6.647751
54397,90493,A,B,B,C,B,B,A,E,E,...,0.293556,0.281192,0.324031,0.051493,0.182182,0.231336,0.176297,0.413605,0.307887,7.423009
99124,165106,A,A,A,C,B,B,A,E,C,...,0.381412,0.427531,0.339106,0.424547,0.446630,0.652907,0.337916,0.374758,0.243413,7.222037
167945,279929,A,A,A,C,B,D,A,E,C,...,0.495404,0.629675,0.281271,0.441942,0.831618,0.710934,0.712174,0.693807,0.269952,8.311413
96789,161220,A,B,B,A,B,D,A,E,E,...,0.734606,0.420961,0.490053,0.368205,0.802769,0.714048,0.354470,0.766466,0.696809,7.667045
110668,184265,A,B,A,C,B,D,A,E,C,...,0.373630,0.464400,0.284551,0.344584,0.776556,0.609952,0.567342,0.713616,0.419782,7.550896
130926,218168,A,B,A,A,B,D,A,B,G,...,0.950352,0.697246,0.752149,0.864242,0.807975,0.546746,0.531759,0.841601,0.302457,8.068698
245046,408735,A,A,A,D,B,B,A,E,C,...,0.385891,0.317842,0.578686,0.422686,0.489252,0.576573,0.651665,0.358783,0.284274,5.174548


# Simple GBM Model

In [20]:
def categorical_convert(X:pd.Series) -> pd.Series:
    temp = X.astype('category').cat.codes
    temp += 1
    return temp

In [21]:
def create_shadow_feature(X:pd.Series) -> np.array:
    temp = X.to_numpy(copy=True)
    np.random.shuffle(temp)
    return temp

In [ ]:
# This is the data frame that we will use to store our SHAP value information
results_df = pd.DataFrame(columns = shadow_features + [target])

for i in range(num_iter):

    X_shadow = X.copy()

    # We will determine optimal parameters on a random sample. We'll use these parameters to construct a new model for each iteration.
    # Determining the hyperparameters takes time, so we'll do it once, and use these parameters to construct our models.
    for f in features:
        X_shadow[f + '_shadow'] = create_shadow_feature(X_shadow[f])

    # These are the features, categorical features, and categorical feature indices that we will use once the shadow features are added.
    shadow_features = features + [f + '_shadow' for f in features]
    shadow_cat_features = cat_features + [x + '_shadow' for x in cat_features]

    X_train, X_test = train_test_split(X_shadow, train_size=sample, shuffle=True)

    params_0 = params_tune(X=X_train,
                           target=target,
                           features=shadow_features,
                           cat_features=shadow_cat_features,
                           num_boost_rounds=num_boost_rounds,
                           params=starting_params,
                           nfolds=nfolds,
                           verbose_eval=False,
                           sample = sample)

    dtrain = lgbm.Dataset(data=X_train[features],
                          label=X_train[target],
                          feature_name=features,
                          categorical_feature= 
                         )

    current_model = lgbm.train(params=params_0,
                               train_set=dtrain,
                               num_boost_round=num_boost_rounds,
                               valid_sets=dval,
                               feature_name=shadow_features,
                               early_stopping_rounds=int(0.05*num_boost_rounds),
                               verbose_eval=100)

    shap_values = shap.TreeExplainer(current_model).shap_values(X_shap)
    shap_values_df = pd.DataFrame(data = shap_values[1], columns = shadow_features)
    shap_values_df[target] = np.where(y_shap == 1, 1, 0)

    print('Missing targets in y_shap = ', sum(y_shap.isna()))
    print('Missing targets in y_shap = ', sum(y_shap))
    print('Missing targets in shap_values_df = ', sum(shap_values_df['target'].isna()))
    print('Missing targets in shap_values_df = ', sum(shap_values_df['target']))

    #print(results.shape)
    #print(np.abs(shap_values[0]).mean(0))
    print(results_df.shape)
    #print(results_df.sample(10))
    print(shap_values_df.shape)
    print([x for x in list(results_df) if x not in list(shap_values_df)])
    print([x for x in list(shap_values_df) if x not in list(results_df)])
    #print(shap_values_df.sample(10))
    results_df = pd.concat([results_df, shap_values_df], axis = 0)

cols_to_keep = []
n = results_df.shape[0]
for col in features:
    x = np.sum(np.where((results_df[target] == 0) & (results_df[col] < results_df[col + '_shadow']), 1,
                    np.where((results_df[target] == 1) & (results_df[col] > results_df[col + '_shadow']), 1, 0)))
    p_val = binom_test(x=x, n=n, p=0.5, alternative='greater')
    if p_val <= 0.05:
        cols_to_keep += [col]

return cols_to_keep



In [22]:
def params_tune(X:pd.DataFrame, target:str, features:list, cat_features:list, num_boost_rounds:int, params:dict, nfolds:int=3, verbose_eval:bool = True, sample:float = 0.5) -> dict:
    
    X = X.sample(frac=sample)
    
    dtrain = lgbm.Dataset(data=X[features],
                          label=X[target],
                          feature_name=features)
    
    model = opt.integration.lightgbm.LightGBMTunerCV(params = params,
                                                     train_set = dtrain,
                                                     num_boost_round = num_boost_rounds,
                                                     nfold = nfolds,
                                                     #stratified = True,
                                                     shuffle = True,
                                                     feature_name=features,
                                                     categorical_feature=cat_features,
                                                     early_stopping_rounds=0.05*num_boost_rounds,
                                                     verbose_eval = verbose_eval,
                                                     seed = 51)
    model.run()
    print(model.best_score)
    return model.best_params

We want to conver the categorical variables to integers.

In [23]:
for f in [x for x in list(data) if 'cat' in x]:
    data[f] = categorical_convert(data[f])

In [ ]:
params = {'objective': 'root_mean_squared_error',
          'verbosity': -1,
          'boosting_type': 'gbdt',
          'learning_rate': 0.25}

params_0 = params_tune(X=data,
                       target='target',
                       features=[x for x in list(data) if ('cat' in x) or ('cont' in x)],
                       cat_features=[x for x in list(data) if ('cat' in x)],
                       num_boost_rounds=5000,
                       params=params,
                       nfolds=3,
                       verbose_eval=False,
                       sample = 1)

In [18]:
def feature_selection_shap(X:pd.DataFrame, features:list, cat_features:list, target:str, starting_params:dict, num_boost_rounds:int=10000, sample:float = 0.7, num_iter:int = 3) -> list:
                            
    # This is the data frame that we will use to store our SHAP value information
    results_df = pd.DataFrame(columns = shadow_features + [target])
    
    for i in range(num_iter):
        
        X_shadow = X.copy()
        
        # We will determine optimal parameters on a random sample. We'll use these parameters to construct a new model for each iteration.
        # Determining the hyperparameters takes time, so we'll do it once, and use these parameters to construct our models.
        for f in features:
            X_shadow[f + '_shadow'] = create_shadow_feature(X_shadow[f])
        
        # These are the features, categorical features, and categorical feature indices that we will use once the shadow features are added.
        shadow_features = features + [f + '_shadow' for f in features]
        shadow_cat_features = cat_features + [x + '_shadow' for x in cat_features]
        
        X_train, X_test = train_test_split(X_shadow, train_size=sample, shuffle=True)
        
        params_0 = params_tune(X=X_train,
                               target=target,
                               features=shadow_features,
                               cat_features=shadow_cat_features,
                               num_boost_rounds=num_boost_rounds,
                               params=starting_params,
                               nfolds=nfolds,
                               verbose_eval=False,
                               sample = sample)
        
        dtrain = lgbm.Dataset(data=X_train[features],
                              label=X_train[target],
                              feature_name=features,
                              categorical_feature= 
                             )
        
        current_model = lgbm.train(params=params_0,
                                   train_set=dtrain,
                                   num_boost_round=num_boost_rounds,
                                   valid_sets=dval,
                                   feature_name=shadow_features,
                                   early_stopping_rounds=int(0.05*num_boost_rounds),
                                   verbose_eval=100)
        
        shap_values = shap.TreeExplainer(current_model).shap_values(X_shap)
        shap_values_df = pd.DataFrame(data = shap_values[1], columns = shadow_features)
        shap_values_df[target] = np.where(y_shap == 1, 1, 0)
        
        print('Missing targets in y_shap = ', sum(y_shap.isna()))
        print('Missing targets in y_shap = ', sum(y_shap))
        print('Missing targets in shap_values_df = ', sum(shap_values_df['target'].isna()))
        print('Missing targets in shap_values_df = ', sum(shap_values_df['target']))
        
        #print(results.shape)
        #print(np.abs(shap_values[0]).mean(0))
        print(results_df.shape)
        #print(results_df.sample(10))
        print(shap_values_df.shape)
        print([x for x in list(results_df) if x not in list(shap_values_df)])
        print([x for x in list(shap_values_df) if x not in list(results_df)])
        #print(shap_values_df.sample(10))
        results_df = pd.concat([results_df, shap_values_df], axis = 0)
    
    cols_to_keep = []
    n = results_df.shape[0]
    for col in features:
        x = np.sum(np.where((results_df[target] == 0) & (results_df[col] < results_df[col + '_shadow']), 1,
                        np.where((results_df[target] == 1) & (results_df[col] > results_df[col + '_shadow']), 1, 0)))
        p_val = binom_test(x=x, n=n, p=0.5, alternative='greater')
        if p_val <= 0.05:
            cols_to_keep += [col]
            
    return cols_to_keep









In [ ]:
features_to_keep = feature_selection_shap(X=data,
                                          features=features,
                                          target='target',
                                          params=params_0,
                                          num_boost_rounds=20000,
                                          sample=0.7,
                                          num_iter=3)

Training until validation scores don't improve for 1000 rounds
[100]	valid_0's auc: 0.615545
[200]	valid_0's auc: 0.620539
[300]	valid_0's auc: 0.622757
[400]	valid_0's auc: 0.621391
[500]	valid_0's auc: 0.62185
[600]	valid_0's auc: 0.622995
[700]	valid_0's auc: 0.624099
[800]	valid_0's auc: 0.622831
[900]	valid_0's auc: 0.625327
[1000]	valid_0's auc: 0.624036
[1100]	valid_0's auc: 0.625355
[1200]	valid_0's auc: 0.626502
[1300]	valid_0's auc: 0.626611
[1400]	valid_0's auc: 0.626031
[1500]	valid_0's auc: 0.626124
[1600]	valid_0's auc: 0.625573
[1700]	valid_0's auc: 0.625923
[1800]	valid_0's auc: 0.625734
[1900]	valid_0's auc: 0.626323
[2000]	valid_0's auc: 0.6267
[2100]	valid_0's auc: 0.627149
[2200]	valid_0's auc: 0.626988
[2300]	valid_0's auc: 0.626752
[2400]	valid_0's auc: 0.626532
[2500]	valid_0's auc: 0.625905
[2600]	valid_0's auc: 0.626303
[2700]	valid_0's auc: 0.626939
[2800]	valid_0's auc: 0.62694
[2900]	valid_0's auc: 0.626564
[3000]	valid_0's auc: 0.627321
Early stopping, best

In [51]:
test_list

['resp']

In [ ]:
dtrain = lgbm.Dataset(data=data[[x for x in features if x not in ['resp','weight']]],
                          label=data['target'],
                          feature_name=[x for x in features if x not in ['resp','weight']])

test_params['metric'] = 'auc'
test_model = lgbm.cv(params = test_params,
                        train_set = dtrain,
                        num_boost_round=10000,
                        nfold=5,
                        stratified=True,
                        shuffle=True,
                        metrics='auc',
                        feature_name=[x for x in features if x not in ['resp','weight']],
                        early_stopping_rounds=100,
                        verbose_eval = 10)

In [ ]:
def num_trees_tune(X:pd.DataFrame, target:str, features:list, num_boost_rounds:int, params:dict, learning_rate:float = 0.01, num_iter:int = 10, train_sample:float = 0.7, verbose_eval:int = 0):
    
    cat_features_indices = [i for i in range(len(features)) if features[i] in cat_features]
    params['learning_rate'] = learning_rate
    best_iter_list = []
        
    for i in range(num_iter):
        X_train, X_val, y_train, y_val = train_test_split(X[features], X[target], train_size = train_sample, shuffle = True, stratify = X[target])
        if len(cat_features) == 0:
            X_train_smote, y_train_smote = SMOTE(random_state=51).fit_resample(X_train, y_train)
        else:
            X_train_smote, y_train_smote = SMOTENC(categorical_features=cat_features_indices, random_state=51).fit_resample(X_train, y_train)
        
        del(X_train, y_train)
        gc.collect()
        
        dtrain = lgbm.Dataset(data = X_train_smote,
                                label = y_train_smote,
                                feature_name = features,
                                categorical_feature = cat_features)
        
        dval = lgbm.Dataset(data = X_val,
                            label = y_val,
                            feature_name = features,
                            categorical_feature = cat_features)
        
        del(X_train_smote, y_train_smote, X_val, y_val)
        gc.collect()
        
        current_model = lgbm.train(params=params,
                                    train_set=dtrain,
                                    num_boost_round=num_boost_rounds,
                                    valid_sets=[dval],
                                    feature_name=features,
                                    categorical_feature=cat_features,
                                    early_stopping_rounds=1000,
                                    verbose_eval=0)

        best_iter_current = current_model.best_iteration
        best_iter_list += [best_iter_current]
        
        print(current_model.best_score)

    return int(np.nanmean(best_iter_list))

In [ ]:
results_df = pd.DataFrame(columns = ['target', 'param', 'val'])

params = {'objective': 'binary',
            'metric': 'binary_logloss',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'learning_rate': 0.25,
            'is_unbalanced': False}
    
for target in targets:
    train = train_all.copy()
    augmented_vals = augmenting_vals(train, target, num = 20)
    train = pd.concat([train, augmented_vals], axis = 0, ignore_index = True)
    best_params = params_tune(X=train, target=target, features=features, cat_features=cat_features, num_boost_rounds=10000, params=params)
    
    features_to_keep = feature_selection_shap(X=train[features], y=train[target], params=best_params, features=features, cat_features=cat_features, sample=0.5, num_boost_rounds=10000, num_iter = 3, verbose_eval = 0)
    cat_features_to_keep = [x for x in cat_features if x in features_to_keep]
    
    best_params['features'] = features_to_keep
    best_params['cat_features'] = cat_features_to_keep
    
    num_trees = num_trees_tune(X=train, target=target, features=features_to_keep, cat_features=cat_features_to_keep, num_boost_rounds=50000, params=best_params, learning_rate = 0.01, num_iter = 10, train_sample = 0.7, verbose_eval = 0)
    best_params['num_trees'] = int(1.1*num_trees)
    
    for key in best_params.keys():
        results_df = pd.concat([results_df, pd.DataFrame(data=[[target, key, best_params[key]]], columns=['target', 'param', 'val'])], axis=0, ignore_index=True)
    print(results_df.tail(10))
    print(target, ' completed. ', len(targets) - targets.index(target) - 1, ' to go...')